In [8]:
import os
import psycopg2
import psycopg2.extras
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder


# Database connection setup
def get_db_connection():
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="HeliosChenThomasBuckley",
        host="db.fxxqwotagugztamftphi.supabase.co",
        port="5432",
    )
    
    print('connected')
    return connection



In [9]:
def fetch_profiles():
    conn = get_db_connection()
    with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as cur:
        cur.execute('SELECT * FROM "Profile" LIMIT 100;')
        profiles = cur.fetchall()
    conn.close()
    return profiles

df = pd.DataFrame(fetch_profiles())


connected


In [12]:
categorical_columns = ['religion', 'body_type', 'education', 'smokes', 'drinks', 'pets', 'diet']

# One-hot encoding for categorical data
encoder = OneHotEncoder(sparse=False)

encoded_data = encoder.fit_transform(df[categorical_columns])


In [13]:
similarity_matrix = cosine_similarity(encoded_data)

# Convert to DataFrame for better readability (optional)
similarity_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)

similarity_df


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.857143,0.571429,0.857143,0.571429,0.714286,0.571429,0.428571,0.714286,0.714286,...,0.857143,0.857143,0.714286,0.428571,0.571429,0.428571,0.285714,0.428571,0.571429,0.571429
1,0.857143,1.000000,0.571429,0.714286,0.571429,0.714286,0.571429,0.428571,0.857143,0.571429,...,1.000000,0.857143,0.571429,0.571429,0.571429,0.428571,0.285714,0.428571,0.428571,0.571429
2,0.571429,0.571429,1.000000,0.714286,0.571429,0.714286,0.714286,0.571429,0.571429,0.428571,...,0.571429,0.714286,0.571429,0.571429,0.571429,0.571429,0.428571,0.285714,0.714286,1.000000
3,0.857143,0.714286,0.714286,1.000000,0.571429,0.857143,0.714286,0.428571,0.571429,0.714286,...,0.714286,0.714286,0.857143,0.571429,0.571429,0.571429,0.428571,0.428571,0.714286,0.714286
4,0.571429,0.571429,0.571429,0.571429,1.000000,0.571429,0.571429,0.571429,0.428571,0.714286,...,0.571429,0.714286,0.428571,0.285714,0.571429,0.285714,0.428571,0.571429,0.571429,0.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.428571,0.428571,0.571429,0.571429,0.285714,0.714286,0.571429,0.428571,0.428571,0.285714,...,0.428571,0.428571,0.714286,0.714286,0.285714,1.000000,0.714286,0.571429,0.428571,0.571429
96,0.285714,0.285714,0.428571,0.428571,0.428571,0.571429,0.428571,0.571429,0.428571,0.428571,...,0.285714,0.285714,0.571429,0.714286,0.142857,0.714286,1.000000,0.714286,0.571429,0.428571
97,0.428571,0.428571,0.285714,0.428571,0.571429,0.571429,0.428571,0.714286,0.285714,0.714286,...,0.428571,0.428571,0.571429,0.428571,0.428571,0.571429,0.714286,1.000000,0.428571,0.285714
98,0.571429,0.428571,0.714286,0.714286,0.571429,0.571429,0.428571,0.714286,0.428571,0.714286,...,0.428571,0.428571,0.571429,0.428571,0.285714,0.428571,0.571429,0.428571,1.000000,0.714286


In [20]:
def recommend_for_user(user_index, top_n=5):
    # Get similarity scores for the user
    user_similarity_scores = similarity_df[user_index]

    # Sort the scores and fetch top N indices
    top_users_indices = user_similarity_scores.sort_values(ascending=False).index[1:top_n+1]

    # Fetch the top recommended users/items
    return df.loc[top_users_indices]

recommendations = recommend_for_user(0, top_n=5)

print(df.loc[0])
print(recommendations.iloc[0])


id                                                         32491
user_id                                                     3200
smokes                                                       yes
orientation                                             straight
body_type                                                average
diet                                                    anything
drinks                                                  socially
education                                             university
height                                                      70.0
job                                                Miscellaneous
location                               san francisco, california
offspring                                             wants kids
pets                                                doesn't have
religion                                           not religious
speaks                                                   english
essay0         hello, my 